In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.nn.init import kaiming_uniform_#
from matplotlib import pyplot as plt
import time

In [2]:
import numpy as np
import gym
from tqdm import tqdm
import random as rand
from itertools import count
from torch.distributions import Categorical
main_seed = 0

In [3]:
env = gym.make("CartPole-v0")
env.seed(0)
env

<TimeLimit<CartPoleEnv<CartPole-v0>>>

In [4]:
class linearApproximator_A2C(nn.Module):
    def __init__(self,state_shape,policy_outputs, state_value_output=1, hidden_dims=(32,32)):
        super(linearApproximator_A2C, self).__init__()
        self.input_size = state_shape
        self.policy_outputs = policy_outputs
        self.state_value_output = state_value_output
        self.device = torch.device("cuda" if torch.cuda.is_available()\
                                   else "cpu")
        
        self.fc1  = nn.Linear(self.input_size,hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_layer = nn.Linear(\
                                hidden_dims[i], hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        
        self.policy_output_layer  = nn.Linear(hidden_dims[-1],self.policy_outputs)
        self.state_value_output_layer = nn.Linear(hidden_dims[-1],self.state_value_output)
        self.to(self.device)
        
        
        
    def forward(self, state_shape):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32)
        state_shape = state_shape.to(self.device)
            
        x = self.fc1(state_shape)
        x = F.relu(x)
        
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        
        logits = self.policy_output_layer(x)#logits, preferences of actions
        state_value = self.state_value_output_layer(x)#predicted state value
        return logits, state_value
        
    def full_pass(self, state):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32)
        state = state.float().to(device)
        logits, state_value = self.forward(state)
        distribution = Categorical(logits=logits)
        action = distribution.sample()#sample action
        log_prob_action = distribution.log_prob(action).unsqueeze(-1)#gets prob of sampled action
        entropy = distribution.entropy().unsqueeze(-1)
        return action.item(), log_prob_action, entropy, logits, state_value

In [1]:
def select_action(policy_network, state, env):
    logits, _ = policy_network.forward(state)
    distribution = Categorical(logits=logits)
    action = distribution.sample()
    action = action.data.numpy().reshape(env.action_space.high.shape)
    return action

In [ ]:
def get_Env_trajectory(env, local_env_seed, A2C_network):
    env.seed(local_env_seed)
    torch.manual_seed(local_env_seed)
    np.random.seed(local_env_seed)
    random.seed(local_env_seed)
    
    abort_after = 60
    start = time.time()
    terminate_env = False
    ovr_rewards = []
    
    while not terminate_env:
        delta = time.time() - start
        

In [ ]:
def A2C_(env,
        n_envs,
        gamma=0.99):
    
    obs_space = len(env.reset())
    action_space = len(env.action_space.high)
    A2C_network = linearApproximator_A2C(obs_space, action_space)
    A2C_network_optimizer = torch.optim.Adam(A2C_network.parameters(),lr=0.0008, weight_decay = 0.01)
    
    for seed in range(n_envs):
        #get trajectory from environment seed using model
        
        #optimize model
        